# Data loading and normalizing

In [ ]:
!pip install pandas numpy hazm scikit-learn gensim transformers

In [6]:
import io

poems = list(line[1:-1] for line in
             io.open('../datasets/shahnameh.txt', mode="r", encoding="utf-8").readlines())

import pandas as pd
import numpy as np
import hazm

# TODO: better stopwords to improve
costums = [
    'زین',
    'مگر',
    'گر',
    'کز',
    'پس',
]
stopwords = set(hazm.stopwords_list() + costums)

poems = np.array(poems)
poems = np.apply_along_axis(' / '.join, 1, poems.reshape(-1, 2))

df = pd.DataFrame(poems, columns=['poems'])

normalizer = hazm.Normalizer(token_based=True)

# find better persian poems normalization and cleaning
def clean_poems(poem):
    tokens = [tk for tk in hazm.word_tokenize(poem) if tk not in stopwords and len(tk) > 1]
    text = ' '.join(tokens)
    return normalizer.normalize(text)

eps = 1e-10

def get_similars_by_cosine_distance(vector, documents, n=5):
    sq_vector = np.squeeze(vector)
    similarity = documents.dot(sq_vector) / (np.linalg.norm(documents, axis=1) * np.linalg.norm(sq_vector) + eps)
    sorted_indx = np.argsort(similarity)
    
    return list(zip(df.poems[sorted_indx[-n:]], similarity[sorted_indx[-n:]]))

df['changed'] = df.poems.apply(clean_poems)

In [2]:
df.sample(10)

,poems,changed
8493,یا تا همه دست نیکی بریم / هان جهان را به بد نسپر,دست نیکی بریم هان جهان بد نسپر
46032,رآنکس که او از در کار بود / دان مرز با او سزاوار,رآنکس کار دان مرز سزاوار
48006,ه جاوید باد آن خردمند مرد / میشه به کام دلش کارک,جاوید باد خردمند مرد میشه کام دلش کارک
27878,ی کین نهان گردد از روی بوم / ود گرز پولاد برسان,کین نهان بوم ود گرز پولاد برسان
41393,دیگر که چندان دلیر و سوار / ه بود اندر ایران همه,چندان دلیر سوار ه_بود اندر ایران
25083,یاده همان شاه دستش بدست / یا و در او را بجای نشس,یاده شاه دستش بدست بجای نشس
41944,رفتند گویندگان نزد شاه / نیده به گفتند زان بی گن,رفتند گویندگان نزد شاه نیده گفتند زان گن
39253,پاکی و از پارسایی زن / ه هم غمگسارست و هم رای ز,پاکی پارسایی زن غمگسارست رای
23457,و بگذشت یک چند گریان چنین / بخشود بر وی جهان آفر,بگذشت گریان بخشود جهان آفر
5259,ر ای دون که تنگ اندر آید سخن / ه جنگ اندرون هیچ,ای دون تنگ اندر آید سخن جنگ اندرون


# Tfidf and Boolean approaches

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('count', CountVectorizer(analyzer='word', ngram_range=(1, 1), max_features=20000)),
                 ('tfidf', TfidfTransformer(sublinear_tf=True))]).fit(df.changed)

def get_document_vectors(series):
    series = [clean_poems(doc) for doc in series]
    boolw_vec = pipe['count'].transform(series).toarray().astype(bool).astype(int)
    norm = np.linalg.norm(boolw_vec, axis=1).reshape(-1, 1)
    return pipe.transform(series).toarray() , boolw_vec / norm

In [5]:
tfidf_documents, boolw_documents = get_document_vectors(df.poems)

In [8]:
tfidf_vector, boolw_vector = \
    get_document_vectors(['که تاج سر شهریاران توی که گوید که پور شبانان توی'])

# TODO: better format to report similarity
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(tfidf_vector, tfidf_documents):
    print("tfidf: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(boolw_vector, boolw_documents):
    print("boolw: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
tfidf: "گهدار ایران نیران توی / ه هر جای پشت دلیران توی" 	 with similarity of 0.39
tfidf: " شاهان گیتی گزیده توی / هانجوی و هم کار دیده توی" 	 with similarity of 0.40
tfidf: "دو گفت سالار و مهتر توی / یاووش رد را برادر توی" 	 with similarity of 0.40
tfidf: "پ لشکر و چنگ شیران توی / گهبان سالار ایران توی" 	 with similarity of 0.41
tfidf: "ه تاج سر شهریاران توی / ه گوید که پور شبانان توی" 	 with similarity of 1.00
----------------------------------------------------------------------------------------------------
boolw: "و کین سر شهریاران بود / ر و کار با تیرباران بود" 	 with similarity of 0.34
boolw: "رانه گزید از بر تاج و گاه / هاده بر خود سر هر سه" 	 with similarity of 0.34
boolw: "کسته سر و پشت پیروزشاه / ه نامداران با تاج و گاه" 	 with similarity of 0.34
boolw: "می رو چنین تا سر کیقباد / ه تاج فریدون به سر بر " 	 with similarity of 0.38
boolw: "ه تاج سر شهریاران توی / ه گوید که پور ش

In [9]:
word_idfs = dict(zip(pipe['count'].get_feature_names_out(), pipe['tfidf'].idf_))

# Combining word embedding and idf

In [10]:
from gensim.models import KeyedVectors
from hazm import word_tokenize

word2vec = KeyedVectors.load_word2vec_format('../models/farsi_literature_word2vec_model.txt')

# TODO: check appropriate stopwords
def embed(poem):
    
    poem = clean_poems(poem)
    def get_wrod2vector(word):
        return word2vec[word] if word in word2vec else np.zeros(100)
    
    embedding_vectors = [get_wrod2vector(wo) * word_idfs.get(wo, 0) for wo in word_tokenize(poem)]
    return np.sum(embedding_vectors, axis=0).tolist()

poems_embeddings = np.array(df.changed.apply(embed).tolist())

In [11]:
sample_embedding = np.array(embed('به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد'))

# TODO: better format to report similarity
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
embedding: "هم آنک از کس ندارد سپاس / ه نیکی وهم نیست یزدان " 	 with similarity of 0.68
embedding: "نین گفت رستم که فرمان شاه / رآنم که برتر ز خورشی" 	 with similarity of 0.68
embedding: "ه دادار گیهان مرا یاورست / ه از دانش برتران برتر" 	 with similarity of 0.72
embedding: "یابد بدو نیز اندیشه راه / ه او برتر از نام و از " 	 with similarity of 0.76
embedding: "ه نام خداوند جان و خرد / زین برتر اندیشه برنگذرد" 	 with similarity of 0.88


# Use BigBird and ParsBert last hidden state as embeddings

In [15]:
from ...models.tran import TransformerEmbedding

ImportError: attempted relative import with no known parent package

In [14]:
embedder = TransformerEmbedding(df)

# Only for the first time
embedder.run()

Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdModel were not initialized from the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj and are newly initialized: ['bert.pooler.

KeyboardInterrupt: 

In [ ]:
with open('../models/embeddings-bigbird.npy', 'rb') as file:
    poems_embeddings = np.load(file)
sample_embedding = tran.get_transformer_embedding(['به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد'], 'bigbird')

# TODO: better format to report similarity
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

In [ ]:
with open('../models/embeddings-parsbert.npy', 'rb') as file:
    poems_embeddings = np.load(file)
sample_embedding = get_transformer_embedding(['چو ضحاک بشنید اندیشه کرد ز خون پدر شد دلش پر ز د'])

# TODO: better format to report similarity
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))